# ItaData2024
propositional example

In [ ]:
using Pkg
Pkg.activate(".")
using MLJ, ModalDecisionTrees
using SoleDecisionTreeInterface, Sole, SoleData
using CategoricalArrays
using DataFrames, JLD2, CSV
using Audio911
using Random
using StatsBase, Catch22
using Test
using Plots

### Settings

In [5]:
sr = 8000
audioparams = (
    sr = sr,
    nfft = 256,
    nbands = 14,
    freq_range = (300, round(Int, sr / 2)),
    db_scale = true,
)

# experiment = :pneumonia
experiment = :bronchiectasis

findhealthy = y -> findall(x -> x == "Healthy", y)
if experiment == :pneumonia
    ds_path = "/datasets/respiratory_Healthy_Pneumonia"
    findsick = y -> findall(x -> x == "Pneumonia", y)
elseif experiment == :bronchiectasis
    ds_path = "/datasets/respiratory_Healthy_Bronchiectasis"
    findsick = y -> findall(x -> x == "Bronchiectasis", y)
else
    error("Unknown type of experiment: $experiment.")
end

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

### Audio features extraction function
This function is called for every audio sample and extract 25 features:
14 bands of the mel spectrogram,
11 spectral features: centroid, crest, entropy, flatness, flux, kurtosis, rolloff, skewness, decrease, slope, spread

In [4]:
function afe(x::AbstractVector{Float64}; sr::Int64, nfft::Int64, nbands::Int64, freq_range::Tuple{Int64, Int64}, db_scale::Bool, get_only_melfreq=false)
    # -------------------------------- parameters -------------------------------- #
    # audio module
    sr = sr
    norm = true
    speech_detection = false
    # stft module
    nfft = nfft
    win_type = (:hann, :periodic)
    win_length = nfft
    overlap_length = round(Int, nfft / 2)
    stft_norm = :power                      # :power, :magnitude, :pow2mag
    # mel filterbank module
    nbands = nbands
    scale = :mel_htk                        # :mel_htk, :mel_slaney, :erb, :bark
    melfb_norm = :bandwidth                 # :bandwidth, :area, :none
    freq_range = freq_range
    # mel spectrogram module
    db_scale = db_scale

    # --------------------------------- functions -------------------------------- #
    # audio module
    audio = load_audio(
        file=x,
        sr=sr,
        norm=norm,
    );

    stftspec = get_stft(
        audio=audio,
        nfft=nfft,
        win_type=win_type,
        win_length=win_length,
        overlap_length=overlap_length,
        norm=stft_norm
    );

    # mel filterbank module
    melfb = get_melfb(
        stft=stftspec,
        nbands=nbands,
        scale=scale,
        norm=melfb_norm,
        freq_range=freq_range
    );

    if get_only_melfreq
        return melfb.data.freq
    end

    # mel spectrogram module
    melspec =  get_melspec(
        stft=stftspec,
        fbank=melfb,
        db_scale=db_scale
    );

    # spectral features module
    spect = get_spectrals(
        source=stftspec,
        freq_range=freq_range
    );

    hcat(
        melspec.spec',
        spect.centroid,
        spect.crest,
        spect.entropy,
        spect.flatness,
        spect.flux,
        spect.kurtosis,
        spect.rolloff,
        spect.skewness,
        spect.decrease,
        spect.slope,
        spect.spread
    );
end

afe (generic function with 1 method)

### Prepare dataset for analysis

In [6]:
color_code = Dict(:red => 31, :green => 32, :yellow => 33, :blue => 34, :magenta => 35, :cyan => 36)
freq = round.(Int, afe(x[1, :audio]; audioparams..., get_only_melfreq=true))
r_select = r"\e\[\d+m(.*?)\e\[0m"

catch9_f = ["max", "min", "mean", "med", "std", "bsm", "bsd", "qnt", "3ac"]
variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz->$j\e[0m" for i in 1:audioparams.nbands]...,
        "\e[$(color_code[:cyan])mcntrd->$j\e[0m", "\e[$(color_code[:cyan])mcrest->$j\e[0m",
        "\e[$(color_code[:cyan])mentrp->$j\e[0m", "\e[$(color_code[:cyan])mflatn->$j\e[0m", "\e[$(color_code[:cyan])mflux->$j\e[0m",
        "\e[$(color_code[:cyan])mkurts->$j\e[0m", "\e[$(color_code[:cyan])mrllff->$j\e[0m", "\e[$(color_code[:cyan])mskwns->$j\e[0m",
        "\e[$(color_code[:cyan])mdecrs->$j\e[0m", "\e[$(color_code[:cyan])mslope->$j\e[0m", "\e[$(color_code[:cyan])msprd->$j\e[0m"
    )
    for j in catch9_f
]...)

catch9 = [
    maximum,
    minimum,
    StatsBase.mean,
    median,
    std,
    Catch22.SB_BinaryStats_mean_longstretch1,
    Catch22.SB_BinaryStats_diff_longstretch0,
    Catch22.SB_MotifThree_quantile_hh,
    Catch22.SB_TransitionMatrix_3ac_sumdiagcov,
]

X = DataFrame([name => Float64[] for name in [match(r_select, v)[1] for v in variable_names]])

audio_feats = [afe(row[:audio]; audioparams...) for row in eachrow(x)]
push!(X, vcat([vcat([map(func, eachcol(row)) for func in catch9]...) for row in audio_feats])...)

yc = CategoricalArray(y)

train_ratio = 0.8

train, test = partition(eachindex(yc), train_ratio, shuffle=true)
X_train, y_train = X[train, :], yc[train]
X_test, y_test = X[test, :], yc[test]

println("Training set size: ", size(X_train), " - ", length(y_train))
println("Test set size: ", size(X_test), " - ", length(y_test))

Training set size: (144, 225) - 144
Test set size: (36, 225) - 36


### Train a model

In [7]:
learned_dt_tree = begin
    Tree = MLJ.@load DecisionTreeClassifier pkg=DecisionTree
    model = Tree(max_depth=-1, )
    mach = machine(model, X_train, y_train)
    fit!(mach)
    fitted_params(mach).tree
end

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/paso/.julia/packages/MLJModels/8W54X/src/loading.jl:159


import MLJDecisionTreeInterface ✔


┌ Info: Training machine(DecisionTreeClassifier(max_depth = -1, …), …).
└ @ MLJBase /home/paso/.julia/packages/MLJBase/7nGJF/src/machines.jl:499


cntrd->std < 205.2
├─ mel14=3539Hz->std < 0.1596
│  ├─ sprd->bsd < 3.5
│  │  ├─ mel3=663Hz->mean < -4.721
│  │  │  ├─ Bronchiectasis (1/1)
│  │  │  └─ Healthy (2/2)
│  │  └─ Bronchiectasis (53/53)
│  └─ flux->std < 2.189
│     ├─ mel10=2106Hz->bsm < 5.5
│     │  ├─ Bronchiectasis (3/3)
│     │  └─ mel12=2749Hz->qnt < 2.153
│     │     ├─ Healthy (20/20)
│     │     └─ mel1=409Hz->bsd < 4.5
│     │        ⋮
│     │        
│     └─ Bronchiectasis (10/10)
└─ crest->3ac < 0.001894
   ├─ Bronchiectasis (2/2)
   └─ Healthy (47/47)


### Model inspection & rule study

In [9]:
sole_dt = solemodel(learned_dt_tree)
# Make test instances flow into the model, so that test metrics can, then, be computed.
apply!(sole_dt, X_test, y_test);
# Print Sole model
printmodel(sole_dt; show_metrics = true, variable_names_map = variable_names);

▣ cntrd->std < 205.24230138276204
├✔ mel14=3539Hz->std < 0.15958429401866944
│├✔ sprd->bsd < 3.5
││├✔ mel3=663Hz->mean < -4.720543021588663
│││├✔ Bronchiectasis : (ninstances = 0, ncovered = 0, confidence = NaN, lift = NaN)
│││└✘ Healthy : (ninstances = 1, ncovered = 1, confidence = 1.0, lift = 1.0)
││└✘ Bronchiectasis : (ninstances = 14, ncovered = 14, confidence = 1.0, lift = 1.0)
│└✘ flux->std < 2.1888435089149336
│ ├✔ mel10=2106Hz->bsm < 5.5
│ │├✔ Bronchiectasis : (ninstances = 1, ncovered = 1, confidence = 1.0, lift = 1.0)
│ │└✘ mel12=2749Hz->qnt < 2.153065633845543
│ │ ├✔ Healthy : (ninstances = 5, ncovered = 5, confidence = 1.0, lift = 1.0)
│ │ └✘ mel1=409Hz->bsd < 4.5
│ │  ├✔ Healthy : (ninstances = 1, ncovered = 1, confidence = 1.0, lift = 1.0)
│ │  └✘ Bronchiectasis : (ninstances = 4, ncovered = 4, confidence = 0.5, lift = 1.0)
│ └✘ Bronchiectasis : (ninstances = 1, ncovered = 1, confidence = 1.0, lift = 1.0)
└✘ crest->3ac < 0.0018940205660926664
 ├✔ Bronchiectasis : (ninstan

### Extract rules that are at least as good as a random baseline model

In [10]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0);
printmodel.(interesting_rules; show_metrics = true, variable_names_map = variable_names);

▣ (cntrd->std < 205.24230138276204) ∧ (mel14=3539Hz->std < 0.15958429401866944) ∧ (sprd->bsd < 3.5) ∧ (¬(mel3=663Hz->mean < -4.720543021588663))  ↣  Healthy : (ninstances = 36, ncovered = 1, coverage = 0.03, confidence = 1.0, lift = 2.12, natoms = 4)
▣ (cntrd->std < 205.24230138276204) ∧ (mel14=3539Hz->std < 0.15958429401866944) ∧ (¬(sprd->bsd < 3.5))  ↣  Bronchiectasis : (ninstances = 36, ncovered = 14, coverage = 0.39, confidence = 1.0, lift = 1.89, natoms = 3)
▣ (cntrd->std < 205.24230138276204) ∧ (¬(mel14=3539Hz->std < 0.15958429401866944)) ∧ (flux->std < 2.1888435089149336) ∧ (mel10=2106Hz->bsm < 5.5)  ↣  Bronchiectasis : (ninstances = 36, ncovered = 1, coverage = 0.03, confidence = 1.0, lift = 1.89, natoms = 4)
▣ (cntrd->std < 205.24230138276204) ∧ (¬(mel14=3539Hz->std < 0.15958429401866944)) ∧ (flux->std < 2.1888435089149336) ∧ (¬(mel10=2106Hz->bsm < 5.5)) ∧ (mel12=2749Hz->qnt < 2.153065633845543)  ↣  Healthy : (ninstances = 36, ncovered = 5, coverage = 0.14, confidence = 1.0, l

### Simplify rules while extracting and prettify result

In [11]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0, normalize = true);
printmodel.(interesting_rules; show_metrics = true, syntaxstring_kwargs = (; threshold_digits = 2), variable_names_map = variable_names);

▣ (cntrd->std < 205.24) ∧ (mel14=3539Hz->std < 0.16) ∧ (sprd->bsd < 3.5) ∧ (mel3=663Hz->mean ≥ -4.72)  ↣  Healthy : (ninstances = 36, ncovered = 1, coverage = 0.03, confidence = 1.0, lift = 2.12, natoms = 4)
▣ (cntrd->std < 205.24) ∧ (mel14=3539Hz->std < 0.16) ∧ (sprd->bsd ≥ 3.5)  ↣  Bronchiectasis : (ninstances = 36, ncovered = 14, coverage = 0.39, confidence = 1.0, lift = 1.89, natoms = 3)
▣ (cntrd->std < 205.24) ∧ (mel14=3539Hz->std ≥ 0.16) ∧ (flux->std < 2.19) ∧ (mel10=2106Hz->bsm < 5.5)  ↣  Bronchiectasis : (ninstances = 36, ncovered = 1, coverage = 0.03, confidence = 1.0, lift = 1.89, natoms = 4)
▣ (cntrd->std < 205.24) ∧ (mel14=3539Hz->std ≥ 0.16) ∧ (flux->std < 2.19) ∧ (mel10=2106Hz->bsm ≥ 5.5) ∧ (mel12=2749Hz->qnt < 2.15)  ↣  Healthy : (ninstances = 36, ncovered = 5, coverage = 0.14, confidence = 1.0, lift = 2.12, natoms = 5)
▣ (cntrd->std < 205.24) ∧ (mel14=3539Hz->std ≥ 0.16) ∧ (flux->std < 2.19) ∧ (mel10=2106Hz->bsm ≥ 5.5) ∧ (mel12=2749Hz->qnt ≥ 2.15) ∧ (mel1=409Hz->bsd < 4

### Directly access rule metrics

In [12]:
readmetrics.(listrules(sole_dt; min_lift=1.0, min_ninstances = 0))

7-element Vector{@NamedTuple{ninstances::Int64, ncovered::Int64, coverage::Float64, confidence::Float64, lift::Float64, natoms::Int64}}:
 (ninstances = 36, ncovered = 1, coverage = 0.027777777777777776, confidence = 1.0, lift = 2.1176470588235294, natoms = 4)
 (ninstances = 36, ncovered = 14, coverage = 0.3888888888888889, confidence = 1.0, lift = 1.894736842105263, natoms = 3)
 (ninstances = 36, ncovered = 1, coverage = 0.027777777777777776, confidence = 1.0, lift = 1.894736842105263, natoms = 4)
 (ninstances = 36, ncovered = 5, coverage = 0.1388888888888889, confidence = 1.0, lift = 2.1176470588235294, natoms = 5)
 (ninstances = 36, ncovered = 1, coverage = 0.027777777777777776, confidence = 1.0, lift = 2.1176470588235294, natoms = 6)
 (ninstances = 36, ncovered = 1, coverage = 0.027777777777777776, confidence = 1.0, lift = 1.894736842105263, natoms = 3)
 (ninstances = 36, ncovered = 9, coverage = 0.25, confidence = 0.8888888888888888, lift = 1.8823529411764706, natoms = 2)

### Show rules with an additional metric (syntax height of the rule's antecedent)

In [13]:
printmodel.(sort(interesting_rules, by = readmetrics); show_metrics = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), variable_names_map = variable_names);

▣ (cntrd->std < 205.24230138276204) ∧ (mel14=3539Hz->std ≥ 0.15958429401866944) ∧ (flux->std ≥ 2.1888435089149336)  ↣  Bronchiectasis : (ninstances = 36, ncovered = 1, coverage = 0.027777777777777776, confidence = 1.0, lift = 1.894736842105263, natoms = 3, height = 2)
▣ (cntrd->std < 205.24230138276204) ∧ (mel14=3539Hz->std ≥ 0.15958429401866944) ∧ (flux->std < 2.1888435089149336) ∧ (mel10=2106Hz->bsm < 5.5)  ↣  Bronchiectasis : (ninstances = 36, ncovered = 1, coverage = 0.027777777777777776, confidence = 1.0, lift = 1.894736842105263, natoms = 4, height = 3)
▣ (cntrd->std < 205.24230138276204) ∧ (mel14=3539Hz->std < 0.15958429401866944) ∧ (sprd->bsd < 3.5) ∧ (mel3=663Hz->mean ≥ -4.720543021588663)  ↣  Healthy : (ninstances = 36, ncovered = 1, coverage = 0.027777777777777776, confidence = 1.0, lift = 2.1176470588235294, natoms = 4, height = 3)
▣ (cntrd->std < 205.24230138276204) ∧ (mel14=3539Hz->std ≥ 0.15958429401866944) ∧ (flux->std < 2.1888435089149336) ∧ (mel10=2106Hz->bsm ≥ 5.5) ∧

### Pretty table of rules and their metrics

In [14]:
metricstable(interesting_rules; variable_names_map = variable_names, metrics_kwargs = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))))

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────┬────────────┬──────────┬───────────┬────────────┬─────────┬────────┬────────┐
│                                                                                                                                                                                                       Antecedent │     Consequent │ ninstances │ ncovered │  coverage │ confidence │    lift │ natoms │ height │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼────────────────┼────────────┼──────────┼───────────┼────────────┼─────────┼────────┼────────┤
│                                                                      (cntrd->

# Inspect features

In [15]:
interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)
printmodel.(interesting_rules; show_metrics = true, syntaxstring_kwargs = (; threshold_digits = 2), variable_names_map=variable_names);

▣ (cntrd->std < 205.24) ∧ (mel14=3539Hz->std < 0.16) ∧ (sprd->bsd ≥ 3.5)  ↣  Bronchiectasis : (ninstances = 36, ncovered = 14, coverage = 0.39, confidence = 1.0, lift = 1.89, natoms = 3)
▣ (cntrd->std < 205.24) ∧ (mel14=3539Hz->std ≥ 0.16) ∧ (flux->std < 2.19) ∧ (mel10=2106Hz->bsm ≥ 5.5) ∧ (mel12=2749Hz->qnt < 2.15)  ↣  Healthy : (ninstances = 36, ncovered = 5, coverage = 0.14, confidence = 1.0, lift = 2.12, natoms = 5)
▣ (cntrd->std ≥ 205.24) ∧ (crest->3ac ≥ 0.0)  ↣  Healthy : (ninstances = 36, ncovered = 9, coverage = 0.25, confidence = 0.89, lift = 1.88, natoms = 2)


In [16]:
interesting_features = unique(SoleData.feature.(SoleLogics.value.(vcat(SoleLogics.atoms.(i.antecedent for i in interesting_rules)...))))
interesting_variables = sort(SoleData.i_variable.(interesting_features))

7-element Vector{Symbol}:
 Symbol("cntrd->std")
 Symbol("crest->3ac")
 Symbol("flux->std")
 Symbol("mel10=2106Hz->bsm")
 Symbol("mel12=2749Hz->qnt")
 Symbol("mel14=3539Hz->std")
 Symbol("sprd->bsd")